In [5]:
import os
import cv2
from PIL import Image
import numpy as np
import albumentations as albu
from albumentations.pytorch import ToTensorV2

In [11]:
def get_training_augmentation():
    train_transform = [

        albu.HorizontalFlip(p=0.5),

        albu.ShiftScaleRotate(scale_limit=2.0, rotate_limit=90, shift_limit=0.1, p=1, border_mode=0),

        albu.PadIfNeeded(min_height=320, min_width=320, always_apply=True, border_mode=0),
        albu.RandomCrop(height=320, width=320, always_apply=True),

        albu.GaussNoise(p=0.2),
        albu.Perspective(p=0.5),

        albu.OneOf(
            [
                albu.CLAHE(p=1),
                albu.RandomBrightness(p=1),
                albu.RandomGamma(p=1),
            ],
            p=0.9,
        ),

        albu.OneOf(
            [
                albu.Sharpen(p=1),
                albu.Blur(blur_limit=3, p=1),
                albu.MotionBlur(blur_limit=3, p=1),
            ],
            p=0.9,
        ),

        albu.OneOf(
            [
                albu.RandomContrast(p=1),
                albu.HueSaturationValue(p=1),
            ],
            p=0.9,
        ),
    ]
    return albu.Compose(train_transform)

In [12]:
augmentation = get_training_augmentation()

for image_file in os.listdir('./images'):
    if image_file.lower().endswith('.jpg'):
        image = Image.open(os.path.join('./images', image_file))
        image = image.resize((320, 320))
        image = image.convert("RGB")
        image = np.array(image)
        
        original = Image.fromarray(image)
        original.save(f"./augmented_images/{image_file}_0.jpg")
        
        for i in range(20):  # 증강된 이미지 10개 생성
            augmented = augmentation(image=image)
            augmented_image = augmented["image"]
            result = Image.fromarray(augmented_image)
            result = result.convert("RGB")
            result.save(f"./augmented_images/{image_file}_{i}.jpg")

TypeError: CLAHE transformation expects 1-channel or 3-channel images.